In [1]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.9/311.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
from neo4j import GraphDatabase

In [3]:
from google.colab import userdata
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver("neo4j+s://7ffc03b9.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [4]:
def insert_data(tx):
    query = """
    CREATE (w:Person {name: 'Walt Whitman'})
    CREATE (p:Person {name: 'Pablo Neruda'})
    CREATE (n:Person {name: 'Nasir Kazmi'})
    CREATE (s:Person {name: 'Sylvia Plath'})
    CREATE (c:Person {name: 'Charles Bukowski'})

    CREATE (w)-[:FRIEND]->(p)
    CREATE (w)-[:FRIEND]->(n)
    CREATE (p)-[:FRIEND]->(n)
    CREATE (n)-[:FRIEND]->(s)
    CREATE (s)-[:FRIEND]->(c)

    RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(insert_data)

In [7]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
import pandas as pd

def show_degree_centrality(driver):
    query = """
      MATCH (p:Person)
      RETURN p.name AS Person,
            size([(p)--() | 1]) AS DegreeCentrality
      ORDER BY DegreeCentrality DESC
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = show_degree_centrality(driver)
pd.DataFrame(results)

,Person,DegreeCentrality
0,Nasir Kazmi,3
1,Pablo Neruda,2
2,Walt Whitman,2
3,Sylvia Plath,2
4,Charles Bukowski,1
